In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_csv('../../orig-power-soton-page-flow.csv')

In [3]:
# Create a new transformed dataset with rows of source and target pages
df_transformed = pd.DataFrame(
    columns = ['user_number', 'session_number', 'session_start_time', 'source', 'source_duration', 'target']
)
column_names = df.columns.values.tolist()
new_row_index = 0
for index, row in df.iterrows():
    for column_name in column_names:
        if 'Unnamed' in column_name and not int(column_name[9:]) % 2 == 0 and pd.notnull(df.loc[index, column_name]):
            column_number = int(column_name[9:])
            if column_number <= 167:
                df_transformed.set_value(new_row_index, 'source', row[column_name])
                df_transformed.set_value(new_row_index, 'source_duration', row[column_name[:9] + str(column_number + 1)])
            if column_number <= 166:
                df_transformed.set_value(new_row_index, 'target', row[column_name[:9] + str(column_number + 2)])
            
            df_transformed.set_value(new_row_index, 'user_number', row['user number'])
            df_transformed.set_value(new_row_index, 'session_number', row['session number'])
            df_transformed.set_value(new_row_index, 'session_start_time', row['session start time'])
            
            new_row_index += 1

In [9]:
import re
s = '12welcome12'
re.sub(r'\d+$', '', s)
df_transformed
aggr_df_transformed = df_transformed.copy()
aggr_df_transformed['source'] = aggr_df_transformed['source'].str.replace(r'\d+$|#', '')
aggr_df_transformed['source'] = aggr_df_transformed['source'].str.replace(r'\d+$', '')
aggr_df_transformed['target'] = aggr_df_transformed['target'].str.replace(r'\d+$|#', '')
aggr_df_transformed['target'] = aggr_df_transformed['target'].str.replace(r'\d+$|#', '')
aggr_df_transformed

,user_number,session_number,session_start_time,source,source_duration,target
0,24015,1,42594.5,homepage,5,registration
1,24015,1,42594.5,registration,27,registrationfinish
2,24015,1,42594.5,registrationfinish,16,usingpower
3,24015,1,42594.5,usingpower,11,registrationfinish
4,24015,1,42594.5,registrationfinish,0,q_followup_8wk
5,24015,1,42594.5,q_followup_8wk,4,q_weight
6,24015,1,42594.5,q_weight,11,q_therapist
7,24015,1,42594.5,q_therapist,10,q_websat
8,24015,1,42594.5,q_websat,5,q_patientenable_
9,24015,1,42594.5,q_patientenable_,6,q_complete_8wk


In [10]:
# dump transformed dataset to csv file
df_transformed.to_csv('./data/power-page-flow-transformed-data.csv', na_rep = 'None', index = False)

In [11]:
# count the number of unique source and target pairs
df_traffic_pairs = df_transformed.groupby(['source', 'target']).size().reset_index().rename(columns={0:'value'})
# df_traffic_pairs = df_traffic_pairs[df_traffic_pairs['value'] > 1]
df_traffic_pairs

,source,target,value
0,FAQheart,FAQspage#5,1
1,FAQpain,s2sendingmessages#2,1
2,FAQspage,linkspage,1
3,FAQspage,loggingout,1
4,FAQspage,weighingFAQ,1
5,FAQspage#1,faqtechnical,1
6,FAQspage#1,loggingout,1
7,FAQspage#1,meettheteam,1
8,FAQspage#1,myprogresspage#1,1
9,FAQspage#2,FAQspage#3,1


In [12]:
# aggregated stuff
aggr_df_traffic_pairs = aggr_df_transformed.groupby(['source', 'target']).size().reset_index().rename(columns={0:'value'})
# aggr_df_traffic_pairs = aggr_df_traffic_pairs[df_traffic_pairs['value'] > 1]
show = aggr_df_traffic_pairs.sort_values('value', ascending=False)
aggr_df_traffic_pairs = aggr_df_traffic_pairs[aggr_df_traffic_pairs['source'] != aggr_df_traffic_pairs['target']]
aggr_df_traffic_pairs.sort_values('value', ascending=False)

# Filtering
aggr_df_traffic_pairs = aggr_df_traffic_pairs[aggr_df_traffic_pairs['value'] > 20]
aggr_df_traffic_pairs.shape

(30, 3)

In [21]:
# Only include nodes that appear in the traffic pairs list
aggr_nodes = pd.DataFrame(columns=['name'])
aggr_nodes['name'] = pd.concat([aggr_df_traffic_pairs['source'], aggr_df_traffic_pairs['target']]).drop_duplicates()
aggr_nodes
print(aggr_nodes)

# save to list
aggr_nodes_list = aggr_nodes.to_dict(orient='record')
print(len(aggr_nodes_list))

                   name
41        chooseeatplan
63           entersteps
103            homepage
113     lastweekgoalspa
155               login
165         loginfailed
169              lowcal
232         pamixchange
237      pawalkbarriers
244        pawalkchange
250         pawalkplans
293        registration
301        reviewfinish
379   s2_s1goaldecision
386     s2_s2goalchange
389       s2_s2newplans
398        s2changeplan
438           s2newgoal
447       s2reviewgoals
490            s2weight
674        stepfeedback
675      stepgoalreview
706         weightgraph
712         welcomeback
715              wplost
169         lowcalideas
293  registrationfinish
301     s2sessionchoice
490              wpgain
29


In [22]:
# store traffic pairs as a list
traffic_pairs_list = df_traffic_pairs.to_dict(orient='record')

In [23]:
# store aggr traffic pairs as a list
aggr_traffic_pairs_list = aggr_df_traffic_pairs.to_dict(orient='record')
len(aggr_traffic_pairs_list)
print(aggr_traffic_pairs_list)

[{'value': 22, 'source': 'chooseeatplan', 'target': 'lowcal'}, {'value': 39, 'source': 'entersteps', 'target': 'stepfeedback'}, {'value': 153, 'source': 'homepage', 'target': 'login'}, {'value': 21, 'source': 'homepage', 'target': 'registration'}, {'value': 23, 'source': 'lastweekgoalspa', 'target': 'reviewfinish'}, {'value': 26, 'source': 'login', 'target': 'loginfailed'}, {'value': 95, 'source': 'login', 'target': 'welcomeback'}, {'value': 23, 'source': 'loginfailed', 'target': 'login'}, {'value': 27, 'source': 'lowcal', 'target': 'lowcalideas'}, {'value': 23, 'source': 'pamixchange', 'target': 'lastweekgoalspa'}, {'value': 22, 'source': 'pawalkbarriers', 'target': 'pawalkplans'}, {'value': 38, 'source': 'pawalkchange', 'target': 'stepgoalreview'}, {'value': 21, 'source': 'pawalkplans', 'target': 'reviewfinish'}, {'value': 21, 'source': 'registration', 'target': 'registrationfinish'}, {'value': 62, 'source': 'reviewfinish', 'target': 's2sessionchoice'}, {'value': 22, 'source': 's2_s1

In [24]:
# count the number times a source page appears (visits)
source_visits = df_transformed.groupby('source').size().reset_index().rename(columns={'source': 'name', 0: 'visits'})
# source_visits = source_visits[source_visits['name'].str.contains('#')]
source_visits

,name,visits
0,FAQheart,1
1,FAQpain,1
2,FAQspage,4
3,FAQspage#1,4
4,FAQspage#2,4
5,FAQspage#3,2
6,FAQspage#4,1
7,FAQspage#5,1
8,Tools,18
9,Tools#1,11


In [25]:
aggr_source_visits = aggr_df_transformed.groupby('source').size().reset_index().rename(columns={'source': 'name', 0: 'visits'})
# source_visits = source_visits[source_visits['name'].str.contains('#')]

aggr_source_visits = aggr_source_visits[aggr_source_visits['visits'] > 30]
# aggr_source_visits.sort_values('visits', ascending=False)
# aggr_source_visits

In [26]:
# store pages and their number of visits as a list 
page_list = source_visits.to_dict(orient='records')

In [27]:
# store aggr pages and their number of visits as a list 
aggr_page_list = aggr_source_visits.to_dict(orient='records')

In [28]:
# create a helper mapper dictionary for querying its index in the page list
nodes_dict_map = {}
for index, page in enumerate(page_list):
    nodes_dict_map[page['name']] = index

In [29]:
# aggr - create a helper mapper dictionary for querying its index in the page list
aggr_nodes_dict_map = {}
for index, page in enumerate(aggr_nodes_list):
    aggr_nodes_dict_map[page['name']] = index
aggr_nodes_dict_map

{'chooseeatplan': 0,
 'entersteps': 1,
 'homepage': 2,
 'lastweekgoalspa': 3,
 'login': 4,
 'loginfailed': 5,
 'lowcal': 6,
 'lowcalideas': 25,
 'pamixchange': 7,
 'pawalkbarriers': 8,
 'pawalkchange': 9,
 'pawalkplans': 10,
 'registration': 11,
 'registrationfinish': 26,
 'reviewfinish': 12,
 's2_s1goaldecision': 13,
 's2_s2goalchange': 14,
 's2_s2newplans': 15,
 's2changeplan': 16,
 's2newgoal': 17,
 's2reviewgoals': 18,
 's2sessionchoice': 27,
 's2weight': 19,
 'stepfeedback': 20,
 'stepgoalreview': 21,
 'weightgraph': 22,
 'welcomeback': 23,
 'wpgain': 28,
 'wplost': 24}

In [30]:
# create a list of dict with source and target represented by their respective index in the page list
traffic_pairs_with_index = []
for pair in traffic_pairs_list:
    traffic_pairs_with_index.append({
        'source': nodes_dict_map[pair['source']],
        'target': nodes_dict_map[pair['target']],
        'value': pair['value']
    })

In [31]:
# aggr - create a list of dict with source and target represented by their respective index in the page list
aggr_traffic_pairs_with_index = []
for pair in aggr_traffic_pairs_list:
    aggr_traffic_pairs_with_index.append({
        'source': aggr_nodes_dict_map[pair['source']],
        'target': aggr_nodes_dict_map[pair['target']],
        'value': pair['value']
    })
aggr_traffic_pairs_with_index

[{'source': 0, 'target': 6, 'value': 22},
 {'source': 1, 'target': 20, 'value': 39},
 {'source': 2, 'target': 4, 'value': 153},
 {'source': 2, 'target': 11, 'value': 21},
 {'source': 3, 'target': 12, 'value': 23},
 {'source': 4, 'target': 5, 'value': 26},
 {'source': 4, 'target': 23, 'value': 95},
 {'source': 5, 'target': 4, 'value': 23},
 {'source': 6, 'target': 25, 'value': 27},
 {'source': 7, 'target': 3, 'value': 23},
 {'source': 8, 'target': 10, 'value': 22},
 {'source': 9, 'target': 21, 'value': 38},
 {'source': 10, 'target': 12, 'value': 21},
 {'source': 11, 'target': 26, 'value': 21},
 {'source': 12, 'target': 27, 'value': 62},
 {'source': 13, 'target': 14, 'value': 22},
 {'source': 13, 'target': 15, 'value': 69},
 {'source': 14, 'target': 15, 'value': 21},
 {'source': 15, 'target': 1, 'value': 26},
 {'source': 16, 'target': 13, 'value': 80},
 {'source': 17, 'target': 16, 'value': 74},
 {'source': 18, 'target': 17, 'value': 85},
 {'source': 19, 'target': 28, 'value': 21},
 {'so

In [32]:
# initial representation of flow dict
flow_dict = {
    'nodes': [{'name': '', 'visits': 0, 'avg_page_duration': 0}],
    'links': [{'source': 0, 'target': 1, 'value': 0, 'user_number': []}]
}

In [33]:
# aggr - initial representation of flow dict
aggr_flow_dict = {
    'nodes': [{'name': '', 'visits': 0, 'avg_page_duration': 0}],
    'links': [{'source': 0, 'target': 1, 'value': 0, 'user_number': []}]
}

In [34]:
flow_dict['nodes'] = page_list
flow_dict['links'] = traffic_pairs_with_index

In [35]:
# aggr
aggr_flow_dict = {}
aggr_flow_dict['nodes'] = aggr_nodes_list
aggr_flow_dict['links'] = aggr_traffic_pairs_with_index

In [36]:
with open('./data/page-flow.json', 'w') as fp:
    json.dump(flow_dict, fp, sort_keys=True, indent=2)

In [37]:
# aggr
with open('./data/aggr-page-flow.json', 'w') as fp:
    json.dump(aggr_flow_dict, fp, sort_keys=True, indent=2)

In [38]:
print(len(aggr_flow_dict['nodes']))
print(len(aggr_flow_dict['links']))

29
30


In [39]:
print(aggr_flow_dict['links'])

[{'value': 22, 'source': 0, 'target': 6}, {'value': 39, 'source': 1, 'target': 20}, {'value': 153, 'source': 2, 'target': 4}, {'value': 21, 'source': 2, 'target': 11}, {'value': 23, 'source': 3, 'target': 12}, {'value': 26, 'source': 4, 'target': 5}, {'value': 95, 'source': 4, 'target': 23}, {'value': 23, 'source': 5, 'target': 4}, {'value': 27, 'source': 6, 'target': 25}, {'value': 23, 'source': 7, 'target': 3}, {'value': 22, 'source': 8, 'target': 10}, {'value': 38, 'source': 9, 'target': 21}, {'value': 21, 'source': 10, 'target': 12}, {'value': 21, 'source': 11, 'target': 26}, {'value': 62, 'source': 12, 'target': 27}, {'value': 22, 'source': 13, 'target': 14}, {'value': 69, 'source': 13, 'target': 15}, {'value': 21, 'source': 14, 'target': 15}, {'value': 26, 'source': 15, 'target': 1}, {'value': 80, 'source': 16, 'target': 13}, {'value': 74, 'source': 17, 'target': 16}, {'value': 85, 'source': 18, 'target': 17}, {'value': 21, 'source': 19, 'target': 28}, {'value': 61, 'source': 19,

In [42]:
aggr_flow_dict['nodes'][22]

{'name': 'weightgraph'}

In [43]:
aggr_flow_dict['nodes'][24]

{'name': 'wplost'}

In [49]:
# find cyclic data
cyclic_links = []
for link in aggr_traffic_pairs_with_index:
    for other_link in aggr_traffic_pairs_with_index:
        if link['target'] == other_link['source'] and other_link['target'] == link['source']:
            if link['value'] > other_link['value']:
                cyclic_links.append(other_link)
print(cyclic_links)

[{'value': 23, 'source': 5, 'target': 4}, {'value': 27, 'source': 22, 'target': 24}]
898


In [52]:
# keep only non-cyclic data
non_cyclic_links = []
for link in aggr_traffic_pairs_with_index:
    same = False
    for cyclic_link in cyclic_links:
        if cyclic_link == link:
            same = True
    if not same:
        non_cyclic_links.append(link)
print(len(non_cyclic_links))

28


In [53]:
non_cyclic_flow_dict = {}
non_cyclic_flow_dict['nodes'] = aggr_nodes_list
non_cyclic_flow_dict['links'] = non_cyclic_links

# non_cyclic
with open('./data/non-cyclic-aggr-page-flow.json', 'w') as fp:
    json.dump(non_cyclic_flow_dict, fp, sort_keys=True, indent=2)